# Convolutaion vision Transformer

## 模型概述

在进入 CvT 之前，我们先简要回顾一下之前章节中讨论的 ViT 架构，以便更好地理解 CvT 架构。ViT 将每幅图像分解为具有固定长度的序列标记（即不重叠的图像块），然后应用多个标准的 Transformer 层，其中包括多头自注意力和位置前馈模块 (FFN)，以建模全局关系进行分类。

卷积视觉 Transformer (CvT) 模型是微软 Cloud+AI 团队 在其论文 [CvT: Introducing Convolutions to Vision Transformers](https://arxiv.org/abs/2103.15808) 中提出的。CvT 结合了 CNN 的所有优点：局部感受野、共享权重、空间下采样，以及 平移、缩放、畸变不变性，同时保留 Transformer 的优点：动态注意力、全局上下文融合、更好的泛化能力。与 ViT 相比，CvT 在保持计算效率的同时实现了更优的性能。此外，由于卷积引入了内建的局部上下文结构，CvT 不再需要位置嵌入，这使其在适应需要可变输入分辨率的广泛视觉任务方面具有潜在优势。

## 模型架构

<div class="wy-nav-content-img">
    <img src="assets/CvT_model_arch.png" width=960px alt="CvT 的模型架构图">
    <p>图1：(a) 整体架构，展示了通过卷积标记嵌入层实现的分层多阶段结构。 (b) 卷积 Transformer 块的详细信息，卷积投影作为第一层。</p>
</div>

上图展示了 CvT 架构的 3 阶段流水线的主要步骤。CvT 的核心在于将两种基于卷积的操作融合到视觉 Transformer 架构中：

* 卷积标记嵌入：将输入图像分割为重叠的图像块，重组为标记，然后输入卷积层。这减少了标记数量（类似于下采样图像中的像素），同时增强其特征丰富度，类似于传统的 CNN。不像其他 Transformer，我们跳过为标记添加预定义的位置信息，而完全依赖卷积操作来捕获空间关系。
* 卷积 Transformer 块：CvT 的每个阶段包含多个此类块。在此，我们使用深度可分离卷积（卷积投影）来处理自注意力模块的“查询”、“键”和“值”组件，而不是 ViT 中的线性投影，如上图所示。这保留了 Transformer 的优点，同时提高了效率。请注意，“分类标记”（用于最终预测）仅在最后一个阶段添加。最后，一个标准的全连接层对最终的分类标记进行分析，以预测图像类别。

## CvT 架构与其他视觉 Transformer 的比较

下表显示了上述代表性并行工作与 CvT 之间在位置编码的必要性、标记嵌入类型、投影类型和主干中的 Transformer 结构方面的关键差异。


| 模型      | 需要位置编码 (PE) | 标记嵌入类型            | 注意力投影类型 | 分层 Transformer |
| --------- | ----------------- | ----------------------- | -------------- | ---------------- |
| ViT, DeiT | 是                | 非重叠                  | 线性           | 否               |
| CPVT      | 否 (带 PE 生成器) | 非重叠                  | 线性           | 否               |
| TNT       | 是                | 非重叠（图像块 + 像素） | 线性           | 否               |
| T2T       | 是                | 重叠（拼接）            | 线性           | 部分 (标记化)    |
| PVT       | 是                | 非重叠                  | 空间缩减       | 是               |
| _CvT_     | _否_              | _重叠（卷积）_          | _卷积_         | _是_             |

## 主要亮点

CvT 实现卓越性能和计算效率的四个主要亮点如下：

* 包含新的 卷积标记嵌入 的 分层 Transformer。
* 利用 卷积投影 的卷积 Transformer 块。
* 由于卷积引入了内建的局部上下文结构， 不需要位置编码。
* 相较于其他视觉 Transformer 架构，参数更少且 FLOPs（每秒浮点运算次数）更低。

## Pytorch 动手实现

In [9]:
import torch
from torch import nn
from einops import rearrange

### 卷积 Token Embedding

In [8]:
class ConvEmbed(nn.Module):
    def __init__(
        self,
        patch_size=7,
        in_chans=3,
        embed_dim=64,
        stride=4,
        padding=2,
        norm_layer=None,
    ):
        super().__init__()
        self.patch_size = patch_size

        self.proj = nn.Conv2d(
            in_chans, embed_dim, kernel_size=patch_size, stride=stride, padding=padding
        )
        self.norm = norm_layer(embed_dim) if norm_layer else None

    def forward(self, x):
        x = self.proj(x)

        B, C, H, W = x.shape
        x = rearrange(x, "b c h w -> b (h w) c")
        if self.norm:
            x = self.norm(x)
        x = rearrange(x, "b (h w) c -> b c h w", h=H, w=W)

        return x

### CvtAttention 中的卷积投射层的实现

<div class="wy-nav-content-img">
    <img src="assets/CvT_conv_projection.png" width=1280px alt="CvT 中的卷积投射层">
    <p>图2：(a) ViT 中的线性投影。 (b) 卷积投影。 (c) 压缩卷积投影（CvT 中的默认设置）</p>
</div>

In [10]:
class CvtSelfAttentionConvProjection(nn.Module):
    def __init__(self, embed_dim, kernel_size, padding, stride):
        super().__init__()
        self.convolution = nn.Conv2d(
            embed_dim,
            embed_dim,
            kernel_size=kernel_size,
            padding=padding,
            stride=stride,
            bias=False,
            groups=embed_dim,
        )
        self.normalization = nn.BatchNorm2d(embed_dim)

    def forward(self, hidden_state):
        hidden_state = self.convolution(hidden_state)
        hidden_state = self.normalization(hidden_state)
        return hidden_state


class CvtSelfAttentionLinearProjection(nn.Module):
    def forward(self, hidden_state):
        batch_size, num_channels, height, width = hidden_state.shape
        hidden_size = height * width
        # rearrange " b c h w -> b (h w) c"
        hidden_state = hidden_state.view(batch_size, num_channels, hidden_size).permute(
            0, 2, 1
        )
        return hidden_state


class CvtSelfAttentionProjection(nn.Module):
    def __init__(
        self, embed_dim, kernel_size, padding, stride, projection_method="dw_bn"
    ):
        super().__init__()
        if projection_method == "dw_bn":
            self.convolution_projection = CvtSelfAttentionConvProjection(
                embed_dim, kernel_size, padding, stride
            )
        self.linear_projection = CvtSelfAttentionLinearProjection()

    def forward(self, hidden_state):
        hidden_state = self.convolution_projection(hidden_state)
        hidden_state = self.linear_projection(hidden_state)
        return hidden_state

### 带卷积的 CvtSelfAttention 实现

In [11]:
class CvtSelfAttention(nn.Module):
    def __init__(
        self,
        num_heads,
        embed_dim,
        kernel_size,
        padding_q,
        padding_kv,
        stride_q,
        stride_kv,
        qkv_projection_method,
        qkv_bias,
        attention_drop_rate,
        with_cls_token=True,
        **kwargs,
    ):
        super().__init__()
        self.scale = embed_dim**-0.5
        self.with_cls_token = with_cls_token
        self.embed_dim = embed_dim
        self.num_heads = num_heads

        self.convolution_projection_query = CvtSelfAttentionProjection(
            embed_dim,
            kernel_size,
            padding_q,
            stride_q,
            projection_method=(
                "linear" if qkv_projection_method == "avg" else qkv_projection_method
            ),
        )
        self.convolution_projection_key = CvtSelfAttentionProjection(
            embed_dim,
            kernel_size,
            padding_kv,
            stride_kv,
            projection_method=qkv_projection_method,
        )
        self.convolution_projection_value = CvtSelfAttentionProjection(
            embed_dim,
            kernel_size,
            padding_kv,
            stride_kv,
            projection_method=qkv_projection_method,
        )

        self.projection_query = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)
        self.projection_key = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)
        self.projection_value = nn.Linear(embed_dim, embed_dim, bias=qkv_bias)

        self.dropout = nn.Dropout(attention_drop_rate)

    def rearrange_for_multi_head_attention(self, hidden_state):
        batch_size, hidden_size, _ = hidden_state.shape
        head_dim = self.embed_dim // self.num_heads
        # rearrange 'b t (h d) -> b h t d'
        return hidden_state.view(
            batch_size, hidden_size, self.num_heads, head_dim
        ).permute(0, 2, 1, 3)

    def forward(self, hidden_state, height, width):
        if self.with_cls_token:
            cls_token, hidden_state = torch.split(hidden_state, [1, height * width], 1)
        batch_size, hidden_size, num_channels = hidden_state.shape
        # rearrange "b (h w) c -> b c h w"
        hidden_state = hidden_state.permute(0, 2, 1).view(
            batch_size, num_channels, height, width
        )

        key = self.convolution_projection_key(hidden_state)
        query = self.convolution_projection_query(hidden_state)
        value = self.convolution_projection_value(hidden_state)

        if self.with_cls_token:
            query = torch.cat((cls_token, query), dim=1)
            key = torch.cat((cls_token, key), dim=1)
            value = torch.cat((cls_token, value), dim=1)

        head_dim = self.embed_dim // self.num_heads

        query = self.rearrange_for_multi_head_attention(self.projection_query(query))
        key = self.rearrange_for_multi_head_attention(self.projection_key(key))
        value = self.rearrange_for_multi_head_attention(self.projection_value(value))

        attention_score = torch.einsum("bhlk,bhtk->bhlt", [query, key]) * self.scale
        attention_probs = torch.nn.functional.softmax(attention_score, dim=-1)
        attention_probs = self.dropout(attention_probs)

        context = torch.einsum("bhlt,bhtv->bhlv", [attention_probs, value])
        # rearrange"b h t d -> b t (h d)"
        _, _, hidden_size, _ = context.shape
        context = (
            context.permute(0, 2, 1, 3)
            .contiguous()
            .view(batch_size, hidden_size, self.num_heads * head_dim)
        )
        return context

## Transformers 中使用

In [3]:
from transformers import AutoFeatureExtractor, CvtForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/cvt-13")
model = CvtForImageClassification.from_pretrained("microsoft/cvt-13")

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# 模型预测 1,000 个 ImageNet 类别中的一个
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: tabby, tabby cat
